<a href="https://colab.research.google.com/github/sumu97/Data-Science-Cheat-Code/blob/main/Kospi_Market_Analysis_sumu97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#코스피 동향분석 및 예측 - sumu97

###개요
 - 코스피 지수의 동향을 예측하여, 선물옵션을 통한 단기적 수익실현 및 투자지표로서의 활용이 가능하도록 합니다.
 - 추후 동일한 모델을 부동산 시장 등에 적용시키기 위해, 데이터에 맞춰 쉽게 변형할 수 있도록 작성합니다.
 - 두 개 이상의 예측 모델을 활용하여 그 결과를 토대로 서로의 예측을 개선할 수 있게 합니다.
 - 아리마 분석으로 Feature 수치를 대략적으로 예측하고, 예측된 데이터를 토대로 Random Forest를 활용해 Target Value를 구합니다.

## 1. 데이터

In [1]:
from google.colab import drive
import io
import pandas as pd

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# 데이터
import datetime

def get_data(NAME,col):
  # 데이터 가져오기
  df = pd.read_csv('drive/MyDrive/Project/Data_For_Kospi/' + NAME + '.csv')

  # 종가, 거래량, 변동% 제외한 Feature 제거 (중복 Issue 가능성)
  df = df.drop(['오픈','고가','저가'], axis=1)

  # 컬럼명 변경 (원본 데이터마다 Columns 다름, 오류처리구문 추가)
  columns_volp = [['date', col + '_' + 'enpr', col + '_' + 'volm', col + '_' + 'chag',]]
  columns_volm = [['date', col + '_' + 'enpr', col + '_' + 'chag',]]
  try:
    df.columns = columns_volp
  except:
    df.columns = columns_volm
  
  # 자료형 변경
  for col in df.columns:
    temp_list = []
    for i in col:
      temp = i.replace('년','')
      temp = temp.replace('월','')
      temp = temp.replace('일','')
      temp = temp.replace(' ','-')
      temp = temp.replace(',','')
      temp = temp.replace('K','000')
      temp = temp.replace('%','')
      try:
        temp = float(temp)
        temp_list.append(temp) # 변경된 자료형을 list에 저장
      except:
        temp = date(temp)



      temp_list.append(temp) # 변경된 자료형을 list에 저장
    df[col] = temp_list # 변경된 리스트를 col 에 저장



  print(df.shape) # 으아아 같은기간 긁었는데 뭐가이렇게 달라
  # 데이터프레임으로 반환
  return df

gd = get_data('GOLD','gd') # 금 시세
#sv = get_data('SILVER','sv') # 은 시세
#ob = get_data('OIL_BRENT','ob') # 브렌트유 시세
#ow = get_data('OIL_WTI','ow') # WTI유 시세
#sp = get_data('S&P','sp') # 미국 증시 S&P 500 지수
#kd = get_data('KOSDAQ','kd') # 코스닥
#kp = get_data('KOSPI','kp') # 코스피(Target)
#kr = get_data('KR10','kr') # 한국 10년 국채 수익률
#us = get_data('US10','us') # 미국 10년 국채 수익률
#jk = get_data('JPY_KRW','jk') # 엔 - 원 환율
#uk = get_data('USD_KRW','uk') # 달러 - 원 환율

(2642, 4)
(3140, 4)
(2604, 4)
(2635, 4)
(2537, 4)
(2485, 4)
(2484, 4)
(3168, 3)
(3012, 3)
(2632, 3)
(2632, 3)


In [43]:
a = '2021'
a = a.replace('년','')
a

'2021'

In [18]:
import datetime
from datetime import date #time#, timedelta
date_time_str = '2018-02-05'
#type(date_time_str)
date = datetime.datetime.strptime(date_time_str, '%Y-%m-%d')
print(date)

AttributeError: ignored

In [40]:
>>> d = datetime.datetime.strptime('2018-05-19', '%Y-%m-%d')
>>> d

,date,gd_enpr,gd_volm,gd_chag
0,2021년 11월 01일,"1,785.55",-,0.24%
1,2021년 10월 31일,"1,781.25",-,-0.15%
2,2021년 10월 29일,"1,783.90",254.51K,-1.04%
3,2021년 10월 28일,"1,802.60",220.88K,0.21%
4,2021년 10월 27일,"1,798.80",182.80K,0.30%
...,...,...,...,...
2637,2011년 10월 07일,"1,634.50",0.07K,-1.05%
2638,2011년 10월 06일,"1,651.90",0.04K,0.71%
2639,2011년 10월 05일,"1,640.30",0.11K,1.59%
2640,2011년 10월 04일,"1,614.70",0.16K,-2.49%


In [34]:
df = pd.merge(gd,sv, on='date')

TypeError: ignored

In [ ]:
df

In [33]:
from functools import reduce
import pandas as pd
dfs = [gd,sv,ob,ow,sp,kd,kp,kr,us,jk,uk]
df = reduce(lambda left, right: pd.merge(left, right, on='date'), dfs)

TypeError: ignored

In [32]:
df = pd.merge([gd,sv,ob,ow,sp,kd,kp,kr,us,jk,uk], on='date')

TypeError: ignored

In [24]:
df

,date,gd_enpr,gd_volm,gd_chag,date,sv_enpr,sv_volm,sv_chag,date,ob_enpr,ob_volm,ob_chag,date,ow_enpr,ow_volm,ow_chag,date,sp_enpr,sp_volm,sp_chag,date,kd_enpr,kd_volm,kd_chag,date,kp_enpr,kp_volm,kp_chag,date,kr_enpr,kr_chag,date,us_enpr,us_chag,date,jk_enpr,jk_chag,date,uk_enpr,uk_chag
0,2021년 11월 01일,"1,785.55",-,0.24%,2021년 11월 01일,23.925,-,0.36%,2021년 11월 01일,83.56,-,0.37%,2021년 11월 01일,83.17,-,0.23%,2021년 10월 29일,"4,605.38",-,0.19%,2021년 11월 01일,998.57,1.16M,0.63%,2021년 11월 01일,"2,978.94",469.02K,0.28%,2021년 11월 01일,2.498,-2.54%,2021년 11월 01일,1.559,-1.11%,2021년 11월 01일,10.2994,0.17%,2021년 11월 01일,"1,177.66",0.26%
1,2021년 10월 31일,"1,781.25",-,-0.15%,2021년 10월 31일,23.840,-,-0.46%,2021년 10월 31일,83.25,-,-1.34%,2021년 10월 31일,82.98,-,-0.71%,2021년 10월 28일,"4,596.42",-,0.98%,2021년 10월 29일,992.33,1.32B,-0.78%,2021년 10월 29일,"2,970.68",545.83M,-1.29%,2021년 10월 31일,2.563,0.00%,2021년 10월 31일,1.577,1.01%,2021년 10월 31일,10.2819,-0.17%,2021년 10월 31일,"1,174.61",0.01%
2,2021년 10월 29일,"1,783.90",254.51K,-1.04%,2021년 10월 29일,23.949,57.80K,-0.71%,2021년 10월 29일,84.38,22.77K,0.07%,2021년 10월 29일,83.57,507.31K,0.92%,2021년 10월 27일,"4,551.68",-,-0.51%,2021년 10월 28일,"1,000.13",1.31B,-0.87%,2021년 10월 28일,"3,009.55",617.26M,-0.53%,2021년 10월 30일,2.563,-0.77%,2021년 10월 29일,1.561,-1.10%,2021년 10월 29일,10.2997,0.14%,2021년 10월 29일,"1,174.47",0.52%
3,2021년 10월 28일,"1,802.60",220.88K,0.21%,2021년 10월 28일,24.120,45.96K,-0.29%,2021년 10월 28일,84.32,118.14K,-0.31%,2021년 10월 28일,82.81,491.59K,0.18%,2021년 10월 26일,"4,574.79",-,0.18%,2021년 10월 27일,"1,008.95",1.27B,-0.28%,2021년 10월 27일,"3,025.49",607.88M,-0.77%,2021년 10월 29일,2.583,8.03%,2021년 10월 28일,1.578,1.81%,2021년 10월 28일,10.2849,-0.10%,2021년 10월 28일,"1,168.44",-0.31%
4,2021년 10월 27일,"1,798.80",182.80K,0.30%,2021년 10월 27일,24.191,48.09K,0.43%,2021년 10월 27일,84.58,149.75K,-2.11%,2021년 10월 27일,82.66,529.95K,-2.35%,2021년 10월 25일,"4,566.48",-,0.47%,2021년 10월 26일,"1,011.76",1.33B,1.75%,2021년 10월 26일,"3,049.08",564.56M,0.94%,2021년 10월 28일,2.391,-3.74%,2021년 10월 27일,1.550,-3.70%,2021년 10월 27일,10.2956,0.64%,2021년 10월 27일,"1,172.05",0.35%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011년 10월 10일,3.840,0.52%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011년 10월 07일,3.820,0.26%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3165,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011년 10월 06일,3.810,0.79%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3166,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011년 10월 05일,3.780,-0.53%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
gd

,date,gd_enpr,gd_volm,gd_chag
0,2021년 11월 01일,"1,785.55",-,0.24%
1,2021년 10월 31일,"1,781.25",-,-0.15%
2,2021년 10월 29일,"1,783.90",254.51K,-1.04%
3,2021년 10월 28일,"1,802.60",220.88K,0.21%
4,2021년 10월 27일,"1,798.80",182.80K,0.30%
...,...,...,...,...
2637,2011년 10월 07일,"1,634.50",0.07K,-1.05%
2638,2011년 10월 06일,"1,651.90",0.04K,0.71%
2639,2011년 10월 05일,"1,640.30",0.11K,1.59%
2640,2011년 10월 04일,"1,614.70",0.16K,-2.49%


In [16]:
kp = kp

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2021년 11월 01일,"2,978.94","2,984.18","2,991.81","2,976.87",469.02K,0.28%
1,2021년 10월 29일,"2,970.68","3,025.67","3,030.17","2,965.40",545.83M,-1.29%
2,2021년 10월 28일,"3,009.55","3,023.17","3,034.42","3,009.55",617.26M,-0.53%
3,2021년 10월 27일,"3,025.49","3,045.83","3,049.02","3,019.00",607.88M,-0.77%
4,2021년 10월 26일,"3,049.08","3,039.82","3,051.65","3,030.53",564.56M,0.94%
...,...,...,...,...,...,...,...
2479,2011년 10월 10일,"1,766.44","1,763.30","1,783.37","1,761.68",274.64M,0.38%
2480,2011년 10월 07일,"1,759.77","1,751.55","1,765.43","1,747.42",334.73M,2.89%
2481,2011년 10월 06일,"1,710.32","1,714.84","1,733.70","1,707.77",352.62M,2.63%
2482,2011년 10월 05일,"1,666.52","1,718.52","1,718.52","1,659.31",336.30M,-2.33%


In [ ]:
f_gold = pd.read_csv('GOLD.csv')

In [ ]:
f_gold = 

In [ ]:
def get_data(csv_name, df_name):
  df_name = pd.read_csv(csv_name)
  
  columns = [[df_name + '_' + 'date', df_name + '_' + 'end_price', df_name + '_' + 'open_price']]

In [ ]:
f_gold

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2021년 11월 01일,"1,785.55","1,781.30","1,788.70","1,781.25",-,0.24%
1,2021년 10월 31일,"1,781.25","1,785.05","1,785.30","1,780.45",-,-0.15%
2,2021년 10월 29일,"1,783.90","1,801.50","1,802.90","1,772.40",254.51K,-1.04%
3,2021년 10월 28일,"1,802.60","1,798.70","1,812.70","1,793.10",220.88K,0.21%
4,2021년 10월 27일,"1,798.80","1,795.00","1,801.00","1,784.30",182.80K,0.30%
...,...,...,...,...,...,...,...
2637,2011년 10월 07일,"1,634.50","1,657.00","1,665.00","1,626.20",0.07K,-1.05%
2638,2011년 10월 06일,"1,651.90","1,642.60","1,654.70","1,637.60",0.04K,0.71%
2639,2011년 10월 05일,"1,640.30","1,629.30","1,644.70","1,599.60",0.11K,1.59%
2640,2011년 10월 04일,"1,614.70","1,661.00","1,678.10","1,596.60",0.16K,-2.49%


In [ ]:
df_name = 'gold'

In [ ]:
  columns = [[df_name + '_' + 'date', df_name + '_' + 'en_price', df_name + '_' + 'op_price', df_name + '_' + 'hi_price',df_name + '_' + 'lo_price',df_name + '_' + 'volume',df_name + '_' + 'change',]]

In [ ]:
columns

[['gold_date',
  'gold_en_price',
  'gold_op_price',
  'gold_hi_price',
  'gold_lo_price',
  'gold_volume',
  'gold_change']]